In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import shutil
import assimilation
import os

from mittag_leffler import ml as ml

In [ ]:
# Plot some functions
alpha = 2.5
fig, axs = plt.subplots(2, 2)
ts = np.linspace(0, 10, 1000)
for alpha, ax in zip([0.5, 1, 1.5, 1.8], axs.flatten()):
  ys = ml(-np.power(ts, alpha), alpha)
  ax.plot(ts, ys, label='$\\alpha = ${}'.format(alpha), linewidth=2)
  ax.set_title('$\\alpha = ${}'.format(alpha))
  ax.set_xlabel('t')
  ax.set_ylabel('x')
  ax.set_ylim(-1.1, 1.1)
plt.tight_layout()
plt.savefig('example-solutions', dpi=600)
plt.show()

In [ ]:
# Plot an example
alpha = 2.5
ts = np.linspace(0, 10, 1000)
dt = ts[1]-ts[0]
z = np.zeros((1, 1, 1000))
z[0, 0, 0] = 1
for i in range(999):
  z[:, :, i+1] = z[:, :, i] + assimilation.step_int(z[:, :, 0:i+1], lambda u : -u, ts[0:i+1], dt, alpha)



fig, ax = plt.subplots(1)
ts = np.linspace(0, 10, 1000)
plt.scatter(ts[::10], z[0, 0, ::10], label='Simulation', marker='+', c='k')

ys = ml(-np.power(ts, alpha), alpha)
ax.plot(ts, ys, label='Exact'.format(alpha), linewidth=2, c='orange')
ax.set_title('Solution of $\\frac{d^{2.5}u}{dt^{2.5}} = -u$', fontsize=20)
ax.set_xlabel('t', fontsize=20)
ax.set_ylabel('u', fontsize=20)
fig.set_size_inches(12, 6)

ax.legend()
plt.show()

In [ ]:
# Investigate errors of the two methods
alpha = 1.8

N_t = np.logspace(1, 4, 10).astype('int')
zs = []
zs_RK = []
z_true = lambda ts : ml(-np.power(ts, alpha), alpha)
err = np.zeros(N_t.shape)
err_RK = np.zeros(N_t.shape)

for (j, N) in enumerate(N_t):
  print('{}/20'.format(j+1))
  ts = np.linspace(0, 15, N)
  dt = ts[1]-ts[0]
  z = np.zeros((1, 1, N))
  z[0, 0, 0] = 1
  z_RK = np.zeros((1, 1, N))
  z_RK[0, 0, 0] = 1
  for i in range(N-1):
    z[:, :, i+1] = z[:, :, i] + assimilation.step_int(z[:, :, 0:i+1], lambda u : -u, ts[0:i+1], dt, alpha)
    z_RK[:, :, i+1] = z_RK[:, :, i] + assimilation.step_int_RK(z_RK[:, :, 0:i+1], lambda u : -u, ts[0:i+1], dt, alpha)
    if i % 100 == 0:
      print(i, end='\r')
  zs.append(z)
  zs_RK.append(z_RK)
  err[j] = np.mean((z - z_true(ts))*(z - z_true(ts)))
  err_RK[j] = np.mean((z_RK - z_true(ts))*(z_RK - z_true(ts)))
plt.plot(15/N_t, err, label='Euler')
plt.plot(15/N_t, err_RK, label='RK4')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Timestep')
plt.ylabel('Error')
plt.gca().invert_xaxis()
plt.legend()
plt.savefig('error', dpi=300)

In [ ]:
# Comparison
fig, axs = plt.subplots(2, 2)

for ax, j in zip(axs.flatten(), [1, 2, 3, -1]):
  ts = np.linspace(0, 15, 1000)
  ax.plot(ts, z_true(ts), label='True', c='k')
  ts = np.linspace(0, 15, N_t[j])
  ax.plot(ts, zs[j][0, 0, :], label='Euler', linestyle='--')
  ax.plot(ts, zs_RK[j][0, 0, :], label='RK4', linestyle='--')
  ax.set_xlabel('t')
  ax.set_ylabel('u')
  ax.set_title('dt={:3.3f}'.format(ts[1]-ts[0]))
fig.set_size_inches(12, 8)
plt.legend()
plt.tight_layout()
plt.savefig('integration', dpi=400)
plt.ylim(-1, 1)
plt.show()